In [5]:
import numpy as np
import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB
import grimsel_config as config
from grimsel import logger
logger.setLevel(0)

In [6]:
mkwargs = {'slct_encar': ['EL'],
           'slct_node': ['CH0', 'DE0', 'SFH_AA', 'SFH_AB'],
           'nhours': {'CH0': (1, 1), 'SFH_AA': (0.25, 0.25), 'SFH_AB': (0.25, 0.5)},
           'slct_pp_type': [],
           'skip_runs': False,
           'tm_filt': [('mt_id', [1, 4, 7, 10]), ('day', [1])],
           'constraint_groups': MB.get_constraint_groups(excl=['chp'])
           }

In [7]:
MB.get_constraint_groups(excl=['hydro'])  # demonstration of the ``get_constraint_groups`` method

['capacity_calculation',
 'capacity_constraint',
 'charging_level',
 'chp',
 'energy_aggregation',
 'energy_constraint',
 'monthly_total',
 'objective',
 'ramp_rate',
 'supply',
 'transmission_bounds',
 'variables',
 'yearly_cost']

In [4]:
iokwargs = {# input
            'sc_inp': None,
            'data_path': config.PATH_CSV,
            # output
            'output_target': 'hdf5',
            'cl_out': 'grimsel_out',
            'no_output': False,
            'resume_loop': False,
            'replace_runs_if_exist': False,
            # general
            'dev_mode': True,
            'sql_connector': None,
            'autocomplete_curtailment': False,
           }

In [5]:
nsteps = [('swco', 3, np.linspace),  # CO2 emission price
          ('swfy', 3, np.arange),    # Future years
         ]

In [6]:
ml = model_loop.ModelLoop(nsteps=nsteps, mkwargs=mkwargs, iokwargs=iokwargs)

In [7]:
ml.init_run_table()
ml.df_def_loop

,run_id,swco_id,swfy_id,swco,swfy,swco_vl,swfy_vl
0,0,0.0,0.0,0.0,0.0,NaN,NaN
1,1,1.0,0.0,0.5,0.0,NaN,NaN
2,2,2.0,0.0,1.0,0.0,NaN,NaN
3,3,0.0,1.0,0.0,1.0,NaN,NaN
4,4,1.0,1.0,0.5,1.0,NaN,NaN
5,5,2.0,1.0,1.0,1.0,NaN,NaN
6,6,0.0,2.0,0.0,2.0,NaN,NaN
7,7,1.0,2.0,0.5,2.0,NaN,NaN
8,8,2.0,2.0,1.0,2.0,NaN,NaN


In [8]:
ml.df_def_loop = ml.df_def_loop.query('swco_id != 2 or swfy_id == 1')
ml.df_def_loop

,run_id,swco_id,swfy_id,swco,swfy,swco_vl,swfy_vl
0,0,0.0,0.0,0.0,0.0,NaN,NaN
1,1,1.0,0.0,0.5,0.0,NaN,NaN
2,2,0.0,1.0,0.0,1.0,NaN,NaN
3,3,1.0,1.0,0.5,1.0,NaN,NaN
4,4,2.0,1.0,1.0,1.0,NaN,NaN
5,5,0.0,2.0,0.0,2.0,NaN,NaN
6,6,1.0,2.0,0.5,2.0,NaN,NaN


In [9]:
ml.io.read_model_data()

In [10]:
ml.m.init_maps()

In [11]:
ml.m.map_to_time_res()

In [15]:
ml.io.write_runtime_tables()

In [16]:
ml.m.get_setlst()
ml.m.define_sets()

Sets up the pyomo set objects as attributes of the :class:`grimsel.core.model_base.ModelBase` class.

The method :func:`grimsel.core.sets.define_sets` generates all necessary pyomo set objects.

The method :func:`grimsel.core.sets.get_setlst` generates a dictionary ``ml.m.setlst`` with the most basic sets, especially those defined by the *set_def_\** columns of the ``ml.m.df_def_plant`` input table:

In [17]:
print(ml.m.setlst['st'])  # for example all storage plants
{pp: ml.m.mps.dict_pp[pp] for pp in ml.m.setlst['st']}  

[57, 60, 102, 103]


{57: 'DE_HYD_STO', 60: 'CH_HYD_STO', 102: 'SFH_AA_STO', 103: 'SFH_AB_STO'}

In [18]:
ml.m.add_parameters()
ml.m.define_variables()
ml.m.add_all_constraints()

In [19]:
ml.m.init_solver()

In [20]:
ml.io.init_output_tables()

In [ ]:
mlm = model_loop_modifier.ModelLoopModifier(ml)
